In [80]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [81]:
df = pd.read_csv("../clean/data_ETLed.csv")

In [82]:
Selected_features = ['Estate_type', 'District', 'Ward', 'Square', 'Numb_bedroom', 'Numb_toilet', 'Numb_floor']
X = df[Selected_features]
y = df['Price']

In [83]:
data_dummies = pd.get_dummies(X, drop_first=True)
data_dummies = data_dummies.astype(float)
cols = data_dummies.columns.values
data_preprocessed = data_dummies[cols]

In [84]:
X_train, X_test, y_train, y_test = train_test_split(data_preprocessed, y, test_size=0.2, random_state=0)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)

In [85]:
model = Ridge()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [86]:
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R2 Score: {r2}')

Mean Squared Error: 6.50066134959502e+23
Mean Absolute Error: 213866740100.53262
R2 Score: -155.42318402490372


In [87]:
perc = np.abs((predictions - y_test)/y_test)
perc = perc.values*100

In [88]:
data = {'Gia du doan': predictions,
        'Gia thuc': y_test,
       '% sai lech': perc}
A = pd.DataFrame(data)
A.sort_values(by=['% sai lech'])

,Gia du doan,Gia thuc,% sai lech
610,1.306484e+10,1.200000e+10,8.873637e+00
381,2.145304e+09,1.750000e+09,2.258878e+01
1050,1.105997e+10,8.600000e+09,2.860430e+01
1281,1.090531e+12,8.250000e+11,3.218554e+01
435,1.096582e+10,7.950000e+09,3.793483e+01
...,...,...,...
1290,-7.527092e+11,4.500000e+09,1.682687e+04
358,-8.870166e+11,4.890000e+09,1.823940e+04
233,1.208384e+13,3.100000e+10,3.888012e+04
980,1.068244e+12,1.650000e+09,6.464208e+04
